In [1]:
%matplotlib inline
%load_ext heat

In [42]:
#%%heat
import multiprocessing as mp
from IPython.display import Image
from copy import copy,deepcopy
from numpy import pi
import numpy as np
from qutip import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit, Gate
import time
from itertools import combinations

def user_gate1():
# S gate
    mat = np.array([[1.,0],[0., 1.j]])
    return Qobj(mat, dims=[[2], [2]])

def user_gate2():
# T gate
    mat = np.array([[1.,0],[0., 0.707+0.707j]])
    return Qobj(mat, dims=[[2], [2]])

def qu(gnum,n): # pick the unitary from the gate set #error
    
    if gnum<2*n+1:
        if gnum%2==0:
            g_t= Gate("T",int((gnum-1)//2)) 
        else:
            g_t= Gate("SNOT", int((gnum-1)//2))
    elif gnum<2*n+n+1:
        #print((gnum-3*n)%n, (gnum-3*n+1)%n )
        g_t= Gate("CNOT",int((gnum-2*n)%n),int((gnum-2*n+1)%n))
    return g_t

def quc(x,n): # generate the given quantum circuit
    qc= QubitCircuit(n)
    for i in range(len(x)):
        if x[i]!=0:
            qc.add_gate(qu(x[i], n))
    prop = qc.propagators()
    return gate_sequence_product(prop).full()
    

#may be faster to apply inverse i.e. loop over numbers gmax**r in base gmax
# def list_circuits(gmax,r):# list the circuits
#     def list_c(*arrays): 
#         grid = np.meshgrid(*arrays)        
#         coord_list = [entry.ravel() for entry in grid]
#         points = np.vstack(coord_list).T
#         return points
#     aa = np.arange(gmax)
#     return list_c(*r*[np.arange(gmax)])

# below is time inefficient but memory efficient
# not optimal 28.8 vs 15.7

def base_conv(cirnum,gmax,r): 
    xx= np.zeros(r)
    num=np.copy(cirnum)
    for i in range(r): 
        xx[i]= num%gmax
        num//=gmax
    return xx



def bi(ii,n):
    bix = np.zeros(n)
    for i in range(n): 
        bix[i]= ii%2
        ii//=2
    return(bix)

def list_povm(n):
    l_povms=[]
    for qbits in range(n+1): 
            for pv in combinations(np.arange(n),n-qbits):
                pov = np.zeros(n)
                for iden in  range(len(pv)):
                    pov[pv[iden]]= 1
                povm = gene_povm(pov,n)
                l_povms.append(np.diag(povm))
    return(l_povms)
                

# below is time inefficient but memory efficient
#removed since, it is less optimal 4.67s vs 3.54s


def gene_povm(gnp,n): 
    pv = np.zeros(2**n)
    b = gnp
    pv[0]= 1
    for j in range(len(b)): 
        if b[j]==1:
            pv[2**j:2**(j+1)]= pv[:2**j]
    return(np.diag(pv)) 

            
                
                

def trqrho(q,rho):
    trq= 0
    trq+= np.sum(np.diag(q)*np.diag(rho),)
    return(trq)

def tr(q): 
    return(np.sum(q))


def opt_povm(r,rho,gmax,n,lst_povm,cnum):
    H=2**n
    if cnum==0:
        counter = 0
        prevtr = 2**n
        thistr = 0
        for povme in range(len(lst_povm)):
            povm= lst_povm[povme]
            trqq = trqrho(rho,povm)
            thistr = tr(povm)
            if trqq>=eta:
                counter+=1
                if thistr<H:
                    H= np.copy(thistr)
            if (thistr!=prevtr) and counter==0:
                prevtr=np.copy(thistr)
                break
            else: 
                prevtr=np.copy(thistr)
    else:
        bc = base_conv(cnum,gmax,r)
        U = quc(bc,n)
        sigma = U@rho@U.conj().T # use of einsum may make it faster? need to make this step faster? sparce?
        counter=0
        prevtr = 2**n
        thistr = 0
        for povme in range(len(lst_povm)):
            povm= lst_povm[povme]
            trqq = trqrho(sigma,povm)
            thistr = tr(povm)
            if trqq>=eta:
                counter+=1
                if thistr<H:
                    H= np.copy(thistr)
            if (thistr!=prevtr) and counter==0:
                prevtr=np.copy(thistr)
                break
            else: 
                prevtr=np.copy(thistr)
    return(H)

    
def H_eep(r,eta,rho,gmax,n):
    ndg_list=[]        
    for cnum in range(0,gmax**r):# this lists out for unique gates
        bc=base_conv(cnum,gmax,r)
        if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):
            ndg_list = np.append(ndg_list,cnum)
    tt = np.nditer(ndg_list)
    lst_povm = list_povm(n)
    nprocs = mp.cpu_count()
    pool=mp.Pool(processes=nprocs)
    H_array= pool.starmap(opt_povm,[(r,rho,gmax,n,lst_povm,circuitnum) for circuitnum in tt])
    pool.close()
    return(np.log(min(H_array))/np.log(2))
    
def H_ee(r,eta,rho,gmax,n):
    ndg_list=[]        
    for cnum in range(0,gmax**r):# this lists out for unique gates
        bc=base_conv(cnum,gmax,r)
        if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):
            ndg_list = np.append(ndg_list,cnum)
            
    lst_povm = list_povm(n)
    circi=0
    povmi=0
    H=2**n
    for cnum in np.nditer(ndg_list):
        if cnum==0:
            counter = 0
            prevtr = 2**n
            thistr = 0
            for povme in range(len(lst_povm)):
                povm= lst_povm[povme]
                trqq = trqrho(rho,povm)
                thistr = tr(povm)
                if trqq>=eta:
                    counter+=1
                    if thistr<H:
                        circi = np.copy(cnum)
                        povmi= povme
                        H= np.copy(thistr)
                if (thistr!=prevtr) and counter==0:
                    prevtr=np.copy(thistr)
                    break
                else: 
                    prevtr=np.copy(thistr)
        else:
            bc = base_conv(cnum,gmax,r)
            U = quc(bc,n)
            sigma = U@rho@U.conj().T # use of einsum may make it faster? need to make this step faster? sparce?
            counter = 0
            prevtr = 2**n
            thistr = 0
            for povme in range(len(lst_povm)):
                povm= lst_povm[povme]
                trqq = trqrho(sigma,povm)
                thistr = tr(povm)
                if trqq>=eta:
                    counter+=1
                    if thistr<H:
                        circi = np.copy(cnum)
                        povmi = povme
                        H= np.copy(thistr)
                if (thistr!=prevtr) and counter==0:
                    prevtr=np.copy(thistr)
                    break
                else: 
                    prevtr=np.copy(thistr)

    print("effective entropy for eta="+str(eta)+" and r="+str(r) +" is "+ str(np.log(H)/np.log(2)))
    print(povmi)
    print(circi)
    return(np.log(H)/np.log(2))

In [43]:
# All the inputs are in this box 

# system specific data
n = 3 # number of qubits
gmax= 1+2*n+n #number of unique gates in the universal gate set [2*n 1-qubit gates, n CNOT gates]
# Need to change qu() if this is changed

# process specific

eta = 0.8 # success rate? 
# define rho below this
x = np.random.rand(2**n,2**n)
rho= x@x.transpose()
rho = rho/np.trace(rho)
#H_ee(r,eta,rho,gmax,n)
np.savetxt("rho2.txt",rho)

In [44]:
eta =0.8
r = 5


In [47]:
%time H_ee(r,eta,rho,gmax,n)
%time H_eep(r,eta,rho,gmax,n)


effective entropy for eta=0.8 and r=5 is 1.0
4
13.0
CPU times: user 3min 4s, sys: 2.23 s, total: 3min 6s
Wall time: 3min 2s
CPU times: user 3.71 s, sys: 154 ms, total: 3.87 s
Wall time: 42.1 s


1.0

In [51]:
eta = 0.80 
rr = np.zeros((8,2))
for i in range(len(rr)):
    r= i
    rho = np.loadtxt("rho2.txt")
    print(np.shape(rho))
    rr[i,0]=i
    rr[i,1]= H_ee(r,eta,rho,gmax,n)
    np.savetxt('rr2.txt',rr)
print(rr)
np.savetxt('rr2.txt',rr)
    
    
    

(8, 8)
effective entropy for eta=0.8 and r=0 is 3.0
[0. 0. 0.]
0
(8, 8)
effective entropy for eta=0.8 and r=1 is 2.0
[1. 1. 0.]
1.0
(8, 8)
effective entropy for eta=0.8 and r=2 is 2.0
[1. 1. 0.]
1.0
(8, 8)
effective entropy for eta=0.8 and r=3 is 2.0
[1. 1. 0.]
1.0
(8, 8)
effective entropy for eta=0.8 and r=4 is 2.0
[1. 1. 0.]
1.0
(8, 8)


KeyboardInterrupt: 

In [50]:
etan = np.zeros((7,2))
x = np.linspace(0.6,0.95,len(etan))
r = 5
for i in range(len(etan)):
    eta = np.copy(x[i])
    etan[i,0]=np.copy(x[i])
    etan[i,1]= H_ee(r,eta,rho,gmax,n)
    np.savetxt("eta1.txt",etan)
np.savetxt("eta1.txt",etan)
    

effective entropy for eta=0.6 and r=5 is 0.0
[0. 0. 0.]
135.0
effective entropy for eta=0.6583333333333333 and r=5 is 0.0
[0. 0. 0.]
135.0
effective entropy for eta=0.7166666666666667 and r=5 is 0.0
[0. 0. 0.]
135.0
effective entropy for eta=0.7749999999999999 and r=5 is 1.0
[1. 0. 0.]
13.0
effective entropy for eta=0.8333333333333333 and r=5 is 2.0
[1. 1. 0.]
1.0
effective entropy for eta=0.8916666666666666 and r=5 is 2.0
[1. 1. 0.]
1.0
effective entropy for eta=0.95 and r=5 is 3.0
[0. 0. 0.]
0


IndexError: index 7 is out of bounds for axis 0 with size 7

In [51]:
len(etan)


7

In [31]:
print(10*((a-1)**(b+1)-1)/(a-2)/a**b)

0.00029881573748535915


In [61]:
a = np.random.rand((55))
b = np.random.rand((55))

In [64]:
%timeit sum(np.random.rand((55))*np.random.rand((55)))
%timeit np.sum(np.random.rand((55))*np.random.rand((55)))


The slowest run took 11.88 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 6.26 µs per loop
The slowest run took 11.18 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 5.84 µs per loop


In [10]:
print(4*np.log(16)/np.log(10))

4.816479930623698


In [63]:
n = 3
qb1= np.random.randint(n)
qb2 = np.random.choice([i for i in range(n) if i!= qb1])
for haarunit in range(100):
    qb1= np.random.randint(n)
    qb2 = np.random.choice([i for i in range(n) if i!= qb1])
    qc1 = QubitCircuit(n)
    qc1.user_gates={"RANDGT": rand_unitary_haar(4, [[2,2],[2,2]])}
    qc1.add_gate("RANDGT",targets=[qb1,qb2])
    props = qc1.propagators()
    V= gate_sequence_product(props).full()
    print(V)


[[ 0.08479546+0.32960981j -0.30558007-0.49067543j  0.        +0.j
   0.        +0.j         -0.17914377-0.27572205j -0.49005108-0.44917741j
   0.        +0.j          0.        +0.j        ]
 [ 0.44145893+0.1735502j   0.45598481+0.41078743j  0.        +0.j
   0.        +0.j         -0.52764156-0.01235508j  0.01424704-0.34578122j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.08479546+0.32960981j
  -0.30558007-0.49067543j  0.        +0.j          0.        +0.j
  -0.17914377-0.27572205j -0.49005108-0.44917741j]
 [ 0.        +0.j          0.        +0.j          0.44145893+0.1735502j
   0.45598481+0.41078743j  0.        +0.j          0.        +0.j
  -0.52764156-0.01235508j  0.01424704-0.34578122j]
 [-0.61335241+0.22574547j  0.30615251+0.35447034j  0.        +0.j
   0.        +0.j          0.07434542+0.07766999j -0.58453513+0.01476622j
   0.        +0.j          0.        +0.j        ]
 [ 0.47906007+0.0500071j   0.02011709+0.263451

In [60]:
Q = rand_unitary_haar(4, [[2,2],[2,2]])

qc1 = QubitCircuit(n)
qc1.user_gates={"RANDGT": rand_unitary_haar(4, [[2,2],[2,2]])}
qc1.add_gate("RANDGT",targets=[1,2])
props = qc1.propagators()
print(gate_sequence_product(props).full())

[[ 0.13961274-0.51708154j -0.37050045+0.40103831j  0.6304565 +0.10431714j
  -0.08160123-0.00405331j  0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.56180766+0.38623742j  0.16986597+0.57587979j -0.10576282-0.10385663j
  -0.38141901+0.0851375j   0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.4609479 +0.12762776j -0.03724294-0.11245367j  0.21613073-0.49314748j
   0.64674557-0.22140136j  0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [-0.11076232-0.08542644j -0.17434353+0.54495613j -0.46045607+0.26027646j
   0.58875501+0.16329448j  0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.13961274-0.51708154j -0.37050045+0.40103831j
   0.6304565 +0.10431714j -0.08160123-0.00405331j]
 [ 0.        +0.j          0.        +0.j    

In [15]:
nprocess=4
def f(x):
    return x**2
def p(t):
    nprocs = mp.cpu_count()
    pool= mp.Pool(processes=nprocs)
    c= pool.map(f,range(t))
    pool.close()
    return(c)
def g(t):
    x= np.arange(t)
    return(f(x))



In [17]:
%time [f(x) for x in range(10000000)]

%time p(10000000)

CPU times: user 2.7 s, sys: 68 ms, total: 2.77 s
Wall time: 2.77 s
CPU times: user 1.1 s, sys: 262 ms, total: 1.36 s
Wall time: 1.37 s


[0,
 1,
 4,
 9,
 16,
 25,
 36,
 49,
 64,
 81,
 100,
 121,
 144,
 169,
 196,
 225,
 256,
 289,
 324,
 361,
 400,
 441,
 484,
 529,
 576,
 625,
 676,
 729,
 784,
 841,
 900,
 961,
 1024,
 1089,
 1156,
 1225,
 1296,
 1369,
 1444,
 1521,
 1600,
 1681,
 1764,
 1849,
 1936,
 2025,
 2116,
 2209,
 2304,
 2401,
 2500,
 2601,
 2704,
 2809,
 2916,
 3025,
 3136,
 3249,
 3364,
 3481,
 3600,
 3721,
 3844,
 3969,
 4096,
 4225,
 4356,
 4489,
 4624,
 4761,
 4900,
 5041,
 5184,
 5329,
 5476,
 5625,
 5776,
 5929,
 6084,
 6241,
 6400,
 6561,
 6724,
 6889,
 7056,
 7225,
 7396,
 7569,
 7744,
 7921,
 8100,
 8281,
 8464,
 8649,
 8836,
 9025,
 9216,
 9409,
 9604,
 9801,
 10000,
 10201,
 10404,
 10609,
 10816,
 11025,
 11236,
 11449,
 11664,
 11881,
 12100,
 12321,
 12544,
 12769,
 12996,
 13225,
 13456,
 13689,
 13924,
 14161,
 14400,
 14641,
 14884,
 15129,
 15376,
 15625,
 15876,
 16129,
 16384,
 16641,
 16900,
 17161,
 17424,
 17689,
 17956,
 18225,
 18496,
 18769,
 19044,
 19321,
 19600,
 19881,
 20164,
 2

In [146]:
pool.close()

In [126]:
def f(axe,aye):
    return(axe**aye)
def g(axe):
    return(axe**aye)

In [6]:
if True==a: 
    print("ahhh")

ahhh


In [127]:
axe=5
aye=2
print(f(axe,aye))
print(g(axe))

25
25


In [ ]:
print(povmi)#873
print(circi)
U = quc(base_conv(circi,gmax,r),n)
print(np.diag(U@rho@U.conj()))
print(np.sum(np.diag(U@rho@U.conj())))